In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from utils.DNV_SN_Curve import DNV_SN_Curve
from utils.extract_and_preprocess_data import extract_and_preprocess_data
from utils.get_forces_and_moments_time_series import get_forces_and_moments_time_series, get_forces_and_moments_time_series_multiprocessed
from utils.get_scf_sector_list import get_scf_sector_list
from utils.calculate_stress_timeseries import calculate_stress_timeseries
from utils.calculate_damage import calculate_damage
from utils.save_damage_table import save_damage_table
import numpy as np
import time
import pandas as pd
import sys # TODO only used for debugging
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


MULTIPROCESS_CASES = True

In [4]:
DLC_IDs = ['DLC12'] #, 'DLC64a', 'DLC64b', 'DLC24a', 'DLC31', 'DLC41a', 'DLC41b']

# Get relevant data_paths for DLC and simulation result files
data_path              = r'C:\Appl\TDI\getting_started\fatpack_test\data'
DLC_file               = data_path +  r'\Doc-0081164-HAL-X-13MW-DGB-A-OWF-Detailed DLC List-Fatigue Support Structure Load Assessment_Rev7.0.xlsx'
sim_res_cluster_folder = data_path +  r'\Doc-0089427-HAL-X-13MW DB-A OWF-ILA3_JLO-model_fatigue_timeseries_all_elevations'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'

geometry      = pd.read_excel(geometry_file).drop([1,2]) # can be used for testing the first geometry
n_geometries  = geometry.shape[0]
point_angles  = [float(i) for i in range(0,359,120)]
cluster_ID    = 'JLO'
curve         = DNV_SN_Curve('D') # DNV SN curve in air, D
out_file_type = 'mat' # or npy 

In [5]:
DLC_ID = DLC_IDs[0]

In [6]:
df, probs, n_cases, n_timesteps = extract_and_preprocess_data(DLC_file, DLC_ID, cluster_ID, sim_res_cluster_folder)

damage_table_DLC_i = np.zeros((n_cases, n_geometries, len(point_angles))) # pre-allocate output matrix of the current DLC

combined_damage_DLC_i = np.zeros((n_geometries, len(point_angles)))

appendix = '_multiprocessed' if MULTIPROCESS_CASES else '_regular'
output_file_name = r'C:\Appl\TDI\getting_started\fatpack_test\fatigue_table' + fr'\fatigue_DB_{cluster_ID}_{DLC_ID}_fatpack{appendix}.{out_file_type}'


In [7]:
geo_idx = 0
geo_row = geometry.iloc[geo_idx]

In [8]:
# For each geometry, points of extra interest wrt stress factors must be identified and the corresponding angles updated
actual_point_angles, scf_per_point = get_scf_sector_list(geo_row, point_angles)

Recalculating angles and SCF


In [113]:
               
# Extract the time series of moments/forces for all points with its probability of occurence and calculate the occuring stress  
forces_timeseries_batch, moments_timeseries_batch = get_forces_and_moments_time_series(df, geo_row, actual_point_angles, n_timesteps, MP=True)

Getting forces and moment timeseries


In [10]:
# Calculate the stress timeseries as a combination of the forces, bending moments, and stress concentration factors and 
stress_timeseries_batch = calculate_stress_timeseries(forces_timeseries_batch, moments_timeseries_batch, geo_row, actual_point_angles, scf_per_point, curve) 


Calculating stress timeseries


In [11]:
# shp = stress_timeseries_batch.shape
# plt.figure()
# plt.plot([i for i in range(shp[2])], stress_timeseries_batch[0, 0, :]) 
# plt.xlim([0,200])          
# plt.show()


In [12]:
# test_ranges = fatpack.find_rainflow_ranges(stress_timeseries_batch[0,0,:], k = 1000)
# print(test_ranges)
# print(test_ranges.shape)

In [116]:
# Find the rainflow ranges and calculate batched 10-min damage for each DLC case for all points/angles the stress timeseries      
damage_geometry_i = calculate_damage(stress_timeseries_batch, curve, MP = True, rainflow_pck = True)

Calculating damage, multiprocessed = True and rainflow_pck = True
Calculating damage multiprocessed


In [117]:
damage_geometry_i

array([[2.77198363e-10, 2.80792374e-09, 8.73721362e-12],
       [5.22963189e-10, 5.90436783e-09, 2.20048226e-11],
       [4.64353514e-10, 4.48643289e-09, 3.43428980e-12],
       ...,
       [1.43798877e-07, 1.28559453e-06, 6.54772306e-09],
       [2.11100088e-07, 1.84899565e-06, 3.90813034e-08],
       [1.24871037e-07, 1.15833277e-06, 6.80214638e-09]])